In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [2]:
data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/train_cleaned.csv")

data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,comment_text_words
0,0000997932d777bf,explanation\n edits made username hardcore ...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '..."
1,000103f0d9cfb60f,daww matches background colour im seemingly ...,0,0,0,0,0,0,"['daww', 'matches', 'background', 'colour', 'i..."
2,000113f07ec002fd,hey man im really trying edit war guy c...,0,0,0,0,0,0,"['hey', 'man', 'im', 'really', 'trying', 'edit..."
3,0001b41b1c6bb37e,cant make real suggestions improvement won...,0,0,0,0,0,0,"['cant', 'make', 'real', 'suggestions', 'impro..."
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page', ..."


# Text Vectorization Layer
### Following [https://www.tensorflow.org/text/tutorials/text_classification_rnn](http://)

In [3]:
NUM_ROWS = 150000
BATCH_SIZE = 64

MAX_TOKENS = 5000
MAX_LENGTH = 200
encoder = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS, output_sequence_length=MAX_LENGTH)
encoder.adapt(data.head(NUM_ROWS)["comment_text"].tolist())

vocab = np.array(encoder.get_vocabulary())
vocab[:20]

len(vocab)

5000

### Building the model

In [7]:
# Sets random seed so results are identical every time
SEED = 1
tf.random.set_seed(SEED)
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

regularization_layer = tf.keras.layers.Dense(
    64, 
    activation="relu", 
    #kernel runs before activation, activity runs after
    kernel_regularizer=tf.keras.regularizers.l1(0.001),
    activity_regularizer=tf.keras.regularizers.l2(0.001)
)

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation="sigmoid")
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=["accuracy"])


### Training the model

In [40]:
# Classification for toxic values ONLY
binaryDf = data.head(NUM_ROWS)[["comment_text", "toxic"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = binaryDf.iloc[:split_cutoff]
validation_data = binaryDf.iloc[split_cutoff:]

training_target = training_data.pop("toxic")
validation_target = validation_data.pop("toxic")

In [8]:
# Classification for all types of toxicity
multiDf = data.head(NUM_ROWS)[["comment_text", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

split_cutoff = int(0.8 * NUM_ROWS)
training_data = multiDf.iloc[:split_cutoff]["comment_text"]
validation_data = multiDf.iloc[split_cutoff:]["comment_text"]

training_target = multiDf.iloc[:split_cutoff][["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
validation_target = multiDf.iloc[split_cutoff:][["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [9]:
# Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2)

history = model.fit(training_data, training_target, epochs=10, validation_data=(validation_data, validation_target), callbacks=[callback], batch_size=BATCH_SIZE)

Epoch 1/10


/opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - ETA: 0s - loss: 0.1259 - accuracy: 0.4944

/opt/conda/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


1875/1875 [==============================] - 110s 49ms/step - loss: 0.1259 - accuracy: 0.4944 - val_loss: 0.0601 - val_accuracy: 0.9940
Epoch 2/10
1875/1875 [==============================] - 87s 47ms/step - loss: 0.0602 - accuracy: 0.7778 - val_loss: 0.0535 - val_accuracy: 0.9940
Epoch 3/10
1875/1875 [==============================] - 87s 47ms/step - loss: 0.0535 - accuracy: 0.8494 - val_loss: 0.0522 - val_accuracy: 0.9940


### Getting test data

In [13]:
# Classification for toxic messages ONLY
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels.loc[test_labels["toxic"] >= 0]
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")

In [16]:
# Classification for all toxicity values
test_data = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test.csv")
test_labels = pd.read_csv("/kaggle/input/toxic-message-classifier-dataset/test_labels.csv")

test_labels = test_labels
merged_df = test_labels.merge(test_data, left_on="id", right_on="id")

In [ ]:
# Tests all rows with a value of 0 or 1 for toxic ONLY

test_df = merged_df[["comment_text", "toxic"]]
testTarget = test_df.pop("toxic")
model.evaluate(test_df, testTarget)

In [18]:
# Tests all rows with a value of 0 or 1 for all toxicity values

test_df = merged_df["comment_text"]
testTarget = merged_df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
model.evaluate(test_df, testTarget)

4787/4787 [==============================] - 86s 18ms/step - loss: -2.7266 - accuracy: 0.9990


[-2.726590156555176, 0.9990010857582092]

In [ ]:
# Tests only rows with a toxic value of 1

test_df = merged_df[["comment_text", "toxic"]]
newTest_df = test_df.loc[test_df["toxic"] == 1]

newTestTarget = newTest_df.pop("toxic")
model.evaluate(newTest_df, newTestTarget)

In [ ]:
# Tests based on validation data

model.evaluate(validation_data, validation_target)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 512)         20398592  
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        295424    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                        

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_8 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, None, 512)         2560000   
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 128)        221952    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 64)               31104     
 nal)                                                            
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                      

In [19]:
submission_set = pd.read_csv('/kaggle/input/toxic-message-classifier-dataset/test.csv')
submission_set.head()

x_test = submission_set['comment_text'].values
y_testing = model.predict(x_test, verbose=1, batch_size=BATCH_SIZE)

 374/2394 [===>..........................] - ETA: 42s

KeyboardInterrupt: 

In [ ]:
submission_df = pd.DataFrame(columns = ['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

submission_df['id'] = submission_set['id']
submission_df['toxic'] = [0 if x < 0 else 1 for x in y_testing]
submission_df['severe_toxic'] = 0.5
submission_df['obscene'] = 0.5
submission_df['threat'] = 0.5
submission_df['insult'] = 0.5
submission_df['identity_hate'] = 0.5

submission_df.head()

submission_df.to_csv('/kaggle/working/submission.csv', index=False)